## Load the csv file and convert features to numeric

In [1]:
# Finds the spark path 
import findspark
findspark.init()

import pyspark
import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

spark = SparkSession.builder \
     .master("local") \
     .appName("hotels") \
     .getOrCreate()

df = spark.read.csv("../input/Hotels_data_Changed.csv", header=True)
# Transform string values to numeric
indexers = [StringIndexer(inputCol="WeekDay", outputCol="WeekDayIndex"),
            StringIndexer(inputCol="Hotel Name", outputCol="HotelNameIndex"),]
pipeline = Pipeline(stages=indexers)
indexed_df = pipeline.fit(df).transform(df)

In [2]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, IntegerType
import pandas as pd

#Transform date values to numeric
dateFormatter = udf(lambda x:  pd.to_datetime(x).toordinal(), IntegerType())

indexed_df = indexed_df.withColumn('SnapshotDateIndex', dateFormatter(col('Snapshot Date')))
indexed_df = indexed_df.withColumn('CheckinDateIndex', dateFormatter(col('Checkin Date')))

### Get the highest discount code

In [3]:
def rowToKeyValue(row):
    key = (row['WeekDayIndex'], row["SnapshotDateIndex"], row["CheckinDateIndex"], row["DayDiff"], row["HotelNameIndex"])
    val = (row['DiscountPerc'], row["Discount Code"])
    return (key,val)

def reduceToMaxPrice(val1, val2):
    price1, code1 = val1
    price2, code2 = val2
    if (price1 > price2):
        return (price1, code1)
    else:
        return (price2, code2)
    
rdd = indexed_df.rdd.map(rowToKeyValue)\
    .reduceByKey(reduceToMaxPrice)\
    .mapValues(lambda val: val[1])

## Create test & training data

In [4]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.regression import LabeledPoint

def mapToLabeledPoint(tup):
    key, val = tup
    # Change range of values from 1-4 to 0-3
    return LabeledPoint(int(val) -1, list(key))

# Split into test and train data
test_data, training_data = rdd.map(mapToLabeledPoint).randomSplit(weights=[0.3, 0.7], seed=1)

## Run decision tree

In [14]:
tree_model = DecisionTree.trainClassifier(training_data, numClasses=4, 
                                          categoricalFeaturesInfo={},
                                          impurity='entropy', maxDepth=5, maxBins=30)

# Print results
print('features 0: Week Day')
print('features 1: Snapshot Date')
print('features 2: Checkin Date')
print('features 3: Day Diff')
print('features 4: Hotel Name')
print(tree_model.toDebugString())

features 0: Week Day
features 1: Snapshot Date
features 2: Checkin Date
features 3: Day Diff
features 4: Hotel Name
DecisionTreeModel classifier of depth 5 with 63 nodes
  If (feature 0 <= 5.0)
   If (feature 4 <= 12.0)
    If (feature 0 <= 4.0)
     If (feature 0 <= 0.0)
      If (feature 4 <= 0.0)
       Predict: 0.0
      Else (feature 4 > 0.0)
       Predict: 3.0
     Else (feature 0 > 0.0)
      If (feature 4 <= 4.0)
       Predict: 1.0
      Else (feature 4 > 4.0)
       Predict: 3.0
    Else (feature 0 > 4.0)
     If (feature 4 <= 9.0)
      If (feature 4 <= 0.0)
       Predict: 3.0
      Else (feature 4 > 0.0)
       Predict: 2.0
     Else (feature 4 > 9.0)
      If (feature 2 <= 735906.0)
       Predict: 0.0
      Else (feature 2 > 735906.0)
       Predict: 1.0
   Else (feature 4 > 12.0)
    If (feature 0 <= 4.0)
     If (feature 0 <= 1.0)
      If (feature 0 <= 0.0)
       Predict: 2.0
      Else (feature 0 > 0.0)
       Predict: 1.0
     Else (feature 0 > 1.0)
      If (feat

## Print decision tree statistics

In [27]:
from pyspark.mllib.evaluation import MulticlassMetrics

predictions = tree_model.predict(test_data.map(lambda p: p.features))
predictionAndLabels = predictions.zip(test_data.map(lambda p: p.label))

metrics = MulticlassMetrics(predictionAndLabels)

print('Accuracy {}'.format(metrics.accuracy))
print('False positive rate {}'.format(metrics.weightedFalsePositiveRate))
print(metrics.confusionMatrix())

Accuracy 0.35379464285714285
False positive rate 0.26186898736862246
DenseMatrix([[1364., 3093., 1656.,  663.],
             [ 873., 6545., 1974.,  836.],
             [ 451., 5419., 2533.,  754.],
             [ 414., 2959., 1752.,  970.]])


## Run naive bayes

In [34]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel

# Train a naive Bayes model.
model = NaiveBayes.train(training_data, 1.0)

# Make prediction
NaiveBayes_predictionAndLabel = test_data.map(lambda p: (float(model.predict(p.features)), p.label))

naive_metrics = MulticlassMetrics(NaiveBayes_predictionAndLabel)

print('Accuracy {}'.format(naive_metrics.accuracy))
print('False positive rate {}'.format(naive_metrics.weightedFalsePositiveRate))
print(naive_metrics.confusionMatrix())

Accuracy 0.259734623015873
False positive rate 0.2240254536536272
DenseMatrix([[  84., 2222.,  208., 4262.],
             [ 103., 3363.,  514., 6248.],
             [  85., 2804.,  551., 5717.],
             [  35., 1479.,  201., 4380.]])
